In [ ]:
import pickle
import os

In [ ]:
os.chdir('../CSE508_Winter2023_Dataset/InvertedIndex')

In [ ]:
#load the invertedindex data
with open('invertedindex.pickle', 'rb') as handle:
    invertedindex = pickle.load(handle)

def get_posting_list(term):
    if term in invertedindex:
        return invertedindex[term][0]
    else:
        return []

def get_posting_listsize(term):
    if term in invertedindex:
        return invertedindex[term][1]

os.chdir('../CSE508_Winter2023_Dataset')
all_docs = os.listdir()

In [ ]:
# and query and number of minimum iterations to find and of two terms
def and_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return and_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)

def and_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            i += 1
        else:
            j += 1
    return result, count


def and_not_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return and_not_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)


def and_not_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
        else:
            j += 1
    while i < size1:
        count+=1
        result.append(posting_list1[i])
        i += 1
    return result, count

def or_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return or_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)


def or_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0

    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
        else:
            result.append(posting_list2[j])
            j += 1
    if i < size1:
        count += 1
        result.extend(posting_list1[i::])
    if j < size2:
        count += 1
        result.extend(posting_list2[j::])
    return result, count




def or_not_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    # all_docs is the list of all documents in the dataset taking difference of alldocs and postinglist2
    # not of postinglist2
    diff = list(set(all_docs) - set(posting_list2))
    #or of posting list1 and not of postinglist2
    result, count = or_query_helper(posting_list1, size1, diff, len(diff))
    return result, count


def or_not_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return or_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)

In [ ]:
#now input and output
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords

In [9]:
# take an input n denoting number of queries
# the next 2*n lines contain queries in the format, input sequence in first line followed by the operations to be performed in next line.

def main():
    n = int(input())
    phrase = []
    ops = []
    for _ in range(n):
        input_sequence = input()
        operations = input()
        phrase.append(input_sequence)
        ops.append(operations)

    for i in range(n):
        query = phrase[i]
        # query = query.split()
        query = query.lower()
        # Perform tokenization
        tokens = word_tokenize(query)
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        # remove punctuations but not hyphen separated words
        tokens = [w for w in tokens if w.isalpha() or '-' in w]
        # Remove blank space tokens
        tokens = [w for w in tokens if w.strip()]
        query = set(tokens)
        op = ops[i]
        op = op.split(',')

        query = list(query)
        result = get_posting_list(query[0])
        size = get_posting_listsize(query[0])
        itr = 0
        TotalCount =0
        # print(op)
        # print(query)
        while(itr<len(op)):
            if op[itr] == 'AND':
                result, count = and_query_helper(result, size, get_posting_list(query[itr+1]), get_posting_listsize(query[itr+1]))
                
                TotalCount += count
                size = len(result)
            elif op[itr] == 'OR':
                # print(query[itr+1])
                result, count = or_query_helper(result, size, get_posting_list(query[itr+1]), get_posting_listsize(query[itr+1]))
                # result = set(result)
                TotalCount += count
                size = len(result)
            elif op[itr] == 'AND NOT':
                # print(query[itr+1])
                result, count = and_not_query_helper(result, size, get_posting_list(query[itr+1]), get_posting_listsize(query[itr+1]))
                # result = set(result)
                TotalCount += count
                size = len(result)
            elif op[itr] == 'OR NOT':
                result, count = or_not_query_helper(result, size, get_posting_list(query[itr+1]), get_posting_listsize(query[itr+1]))
                # result = set(result)
                TotalCount += count
                size = len(result)
            itr += 1
        string = query[0]
        for i in range(1,len(query)):
            string += ' '+op[i-1]+' '+query[i]
        print("Query ",n+1,": ",string)
        print("Number of documents retrieved for query ",n+1,": ",len(result))
        print("Names of the documents retrieved for query ",n+1,": ",result)
        print("Number of comparisons required for query ",n+1,": ",TotalCount)

        
main()

Query  2 :  fluid AND velocity
Number of documents retrieved for query  2 :  65
Names of the documents retrieved for query  2 :  ['cranfield0004', 'cranfield0014', 'cranfield0044', 'cranfield0059', 'cranfield0061', 'cranfield0071', 'cranfield0072', 'cranfield0073', 'cranfield0087', 'cranfield0088', 'cranfield0106', 'cranfield0112', 'cranfield0128', 'cranfield0131', 'cranfield0133', 'cranfield0139', 'cranfield0151', 'cranfield0191', 'cranfield0241', 'cranfield0257', 'cranfield0260', 'cranfield0267', 'cranfield0268', 'cranfield0269', 'cranfield0296', 'cranfield0299', 'cranfield0321', 'cranfield0351', 'cranfield0352', 'cranfield0397', 'cranfield0414', 'cranfield0417', 'cranfield0452', 'cranfield0460', 'cranfield0474', 'cranfield0475', 'cranfield0477', 'cranfield0550', 'cranfield0559', 'cranfield0653', 'cranfield0770', 'cranfield0771', 'cranfield0773', 'cranfield0787', 'cranfield0788', 'cranfield0872', 'cranfield0962', 'cranfield0963', 'cranfield0967', 'cranfield0987', 'cranfield1072', 'cr